# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# Imports

In [ ]:
from search_eval.eval_generic import SGLDES
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *

from nni import trace
import nni.retiarii.strategy as strategy
import nni.retiarii.serializer as serializer

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader

import torch

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# Configuration: 

### Search == One-Shot
### SGLDES == True

In [ ]:


from search_space.unet.unetspaceOS import UNetSpace

total_iterations = 650

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
img_id = np.random.randint(0, 50)


phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

learning_rate = 0.095
buffer_size = 100
patience = 250
weight_decay = 5e-7
show_every = 200
report_every = 25

# Create the lightning module
module = SGLDES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=learning_rate, # consider .01
                buffer_size=buffer_size,
                patience=patience,
                weight_decay= weight_decay,

                show_every=show_every,
                report_every=report_every,
                HPO=False,
                NAS=True,
                OneShot=True,
                SGLD_regularize=False,
                ES=True,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom_noisy, num_iter=1), batch_size=1)
val_loader = DataLoader(SingleImageDataset(phantom_noisy, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Select the Search Strategy
# search_strategy = strategy.DARTS()
# search_strategy = strategy.ENAS()
search_strategy = strategy.GumbelDARTS()
# search_strategy = strategy.RandomOneShot()


# model_space = UNetSpace(
#          C_in=1, 
#          C_out=1, 
#          depth=2, 
#          nodes_per_layer=1, # accept only 1 or 2,
#          ops_per_node=4,
#          use_attention=False,

#         )

model_space = UNetSpace(
         C_in=1, 
         C_out=1, 
         depth=4, 
         nodes_per_layer=2, # accept only 1 or 2,
         ops_per_node=4,
         use_attention=True,

        )
# fast_dev_run=False

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=search_strategy)
experiment.run(config)

In [ ]:
experiment.export_top_models()[0]

In [ ]:
# retrain top model
from search_space.node_space import exportedModel
from search_space.unet import exportedUNet
from search_eval.eval_no_search_SGLD_ES import Eval_SGLD_ES

# construct output and retrain
exported_arch = experiment.export_top_models()[0]
# extract value from key -- pool 0
print("--------------------")
print("--------------------")
for i in range(depth):
    print(f'pool {i+1}: ', exported_arch[f'pool {i}/op_1_0'])
    print(f'encoder {i+1}: ', exported_arch[f'encoder {i}/op_1_0'])
    print("--------------------")
for i in range(depth):
    print(f'upsample {i+1}: ', exported_arch[f'upsample {i}/op_1_0'])
    print(f'decoder {i+1}: ', exported_arch[f'decoder {i}/op_1_0'])
    print("--------------------")
print("--------------------\n\n\n")

model = exportedModel(1,1,depth,exported_arch)

learning_rate = 0.01
buffer_size = 100
patience = 1500
weight_decay = 5e-7
show_every = 500
report_every = 50

print(f"\n\n----------------------------------")
print(f'Experiment Configuration:')
print(f'\tTotal Iterations: {total_iterations}')

print(f'\tPatience: {patience}')
print(f'\tBuffer Size: {buffer_size}')
print(f'\tLearning Rate: {learning_rate}')
print(f'\tWeight Decay: {weight_decay}')

print(f'\tImage Resolution: {resolution}')
print(f'\tPlotting every {show_every} iterations')
print(f"----------------------------------\n\n")

# Create the lightning module
module = Eval_SGLD_ES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,

                learning_rate=learning_rate, 
                buffer_size=buffer_size,
                patience=patience,
                weight_decay=weight_decay,
                
                report_every=report_every,
                show_every=show_every,
                model=model,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []

# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)
lightning.fit(model)

In [ ]:
# stop experiment and clear cache
experiment.stop()
torch.cuda.empty_cache()

# Configuration: 

### Search == Multi-Trial
### SGLDES == True

In [ ]:
from search_space.unet.unetspaceMT import UNetSpaceMT
total_iterations = 800

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'

img_id = 45 # np.random.randint(0, 50)
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

learning_rate = 0.1
buffer_size = 100
patience = 200
weight_decay = 5e-8
show_every = 200
report_every = 25

print(f"\n\n----------------------------------")
print(f'Experiment Configuration:')
print(f'\tImage Config')
print(f'\t\tImage ID: {img_id}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tNoise Type: {noise_type}')
print(f'\t\tNoise Level: {noise_level}\n')

print(f'\tTraining Config')
print(f'\t\tTotal Iterations: {total_iterations}')
print(f'\t\tBuffer Size: {buffer_size}')
print(f'\t\tPatience: {patience}')
print(f'\t\tLearning Rate: {learning_rate}')
print(f'\t\tWeight Decay: {weight_decay}')
print(f'\t\tPlotting every {show_every} iterations\n')

print(f"----------------------------------\n\n")


# Create the lightning module
module = SGLDES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=learning_rate, # consider .01
                buffer_size=buffer_size,
                patience=patience,
                weight_decay= weight_decay,

                show_every=show_every,
                report_every=report_every,
                HPO=True,
                NAS=True,
                OneShot=False,
                SGLD_regularize=True,
                switch=None
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )

if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)

# Create a Search Space
model_space = UNetSpaceMT(depth=4, init_features=64)

# Select a Search Strategy
# search_strategy = strategy.Random(dedup=True)
# search_strategy = strategy.TPE()
search_strategy = strategy.RegularizedEvolution(dedup=True)


# Configure and run the experiment for multi-strategy
experiment = RetiariiExperiment(model_space, lightning, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = f'MT_{search_strategy.__class__.__name__}_p{img_id}_NT{noise_type}_NL{noise_level}'

exp_config.max_trial_number = 250   # spawn 4 trials at most
exp_config.trial_concurrency = 3  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

experiment.run(exp_config, 8889) # recommended in documentation was 8081
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=search_strategy)
experiment.run(config)

In [ ]:
experiment.stop()

# Configuration:

### Search == None
### SGLDES == True

In [ ]:
from search_space.unet.unet import UNet

# INPUTS
# Non HPO inputs
total_iterations = 1000
show_every = 100 #200
report_every = 25 #25

# HPO inputs
#  note a smaller learning rate affecs the SGLD, so overfitting happens FASTER at LOWER learning rates (start with 0.01)
learning_rate = .1 # 0.1089
buffer_size = 100 # 500
patience = 100
weight_decay= 5e-8

resolution = 64
noise_level = 0.09
noise_type = 'gaussian'
img_id = np.random.randint(0, 50) # 45

phantom = np.load(f'phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy= np.load(f'phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

# model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=64, pretrained=False)
model = UNet(depth=4, init_features=64)


print(f"\n\n----------------------------------")
print(f'Experiment Configuration:')
print(f'\tImage Config')
print(f'\t\tImage ID: {img_id}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tNoise Type: {noise_type}')
print(f'\t\tNoise Level: {noise_level}\n')

print(f'\tTraining Config:')
print(f'\t\tTotal Iterations: {total_iterations}')
print(f'\t\tBuffer Size: {buffer_size}')
print(f'\t\tPatience: {patience}')
print(f'\t\tLearning Rate: {learning_rate}')
print(f'\t\tWeight Decay: {weight_decay}\n')

print(f'\tLogging Config:')
print(f'\t\tReport every: {report_every}')
print(f'\t\tPlotting every {show_every} iterations\n')
print(f"----------------------------------\n\n")

# Create the lightning module
module = SGLDES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=learning_rate,
                buffer_size=buffer_size,
                patience=patience,
                weight_decay= weight_decay,

                show_every=show_every,
                report_every=report_every,
                HPO=False,
                NAS=False,
                OneShot=False,
                SGLD_regularize=True,
                switch=None
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)
lightning.fit(model)

In [ ]:
# Create the lightning module
### this one works but we want to use the universal one if we can
# module = Eval_SGLD_ES(
#                 phantom=phantom, 
#                 phantom_noisy=phantom_noisy,

#                 learning_rate=learning_rate, 
#                 buffer_size=buffer_size,
#                 patience=patience,
#                 weight_decay=weight_decay,
                
#                 report_every=report_every,
#                 show_every=show_every,
#                 model=model,
#                 )

# Configuration:

### Search == None
### SGLDES == False (SGLD)

In [ ]:
from search_eval.utils.common_utils import *
from search_eval.eval_SGLD import Eval_SGLD, SingleImageDataset

from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from search_space.unet.unet import UNet

import numpy as np
import torch
torch.cuda.empty_cache()

dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# INPUTS
total_iterations = 1400
burnin_iter = 200
show_every = 50
lr = 0.1 #  note a smaller learning rate affecs the SGLD, so overfitting happens FASTER at LOWER learning rates (start with 0.01)

model = UNet(in_channels=1, out_channels=1, init_features=64, depth=4)
# model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=64, pretrained=False)

img_id = np.random.randint(0, 50)
resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

print(f"\n\n----------------------------------")
print(f'Experiment Configuration:')
print(f'\tImage Config')
print(f'\t\tImage ID: {img_id}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tNoise Type: {noise_type}')
print(f'\t\tNoise Level: {noise_level}\n')

print(f'\tModel Config')
print(f'\t\tModel: {model}\n')

print(f'\tTraining Config')
print(f'\t\tTotal Iterations: {total_iterations}')
print(f'\t\tBurnin Iterations: {burnin_iter}')
print(f'\t\tLearning Rate: {lr}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tPlotting every {show_every} iterations\n')
print(f"----------------------------------\n\n")
# Create the lightning module
module = Eval_SGLD(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                lr=lr, 
                burnin_iter=burnin_iter,
                model=model, # model defaults to U-net 
                show_every=show_every
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)
lightning.fit(model)